In [2]:
import os
import json
import datasets

# ds = datasets.load_dataset("yunusskeete/Carla-COCO-Object-Detection-Dataset-No-Images")
ds = datasets.load_dataset("yunusskeete/Carla-COCO-Object-Detection-Dataset")
ds

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [47]:
logger = datasets.logging.get_logger(__name__)

_DESCRIPTION = """\
This dataset contains 1028 images each 640x380 pixels.
The dataset is split into 249 test and 779 training examples.
Every image comes with MS COCO format annotations.
The dataset was collected in Carla Simulator, driving around in autopilot mode in various environments
(Town01, Town02, Town03, Town04, Town05) and saving every i-th frame.
The labels where then automatically generated using the semantic segmentation information.
"""

_HOMEPAGE = "https://github.com/yunusskeete/Carla-COCO-Object-Detection-Dataset"

_LICENSE = "MIT"

_URL = "https://github.com/yunusskeete/Carla-COCO-Object-Detection-Dataset/raw/master/Carla-COCO-Object-Detection-Dataset-No-Images/"
_URLS = {
    "train": _URL + "train/train.tar.gz",
    "test": _URL + "test/test.tar.gz",
}

_CATEGORIES = ["automobile", "bike", "motorbike", "traffic_light", "traffic_sign"]

def _info(self):
    """This method specifies the datasets.DatasetInfo object which contains informations and typings for the dataset"""

    features = datasets.Features(
        {
            "image_id": datasets.Value("int64"),
            # "image": datasets.Image(),
            "width": datasets.Value("int32"),
            "height": datasets.Value("int32"),
            "file_name": datasets.Value("string"),
            "url": datasets.Value("string"),
            "objects": datasets.Sequence(
                {
                    "id": datasets.Sequence(datasets.Value("int64")),
                    "area": datasets.Sequence(datasets.Value("int64")),
                    "bbox": datasets.Sequence(datasets.Value("float32"), length=4),
                    "category": datasets.Sequence(datasets.ClassLabel(names=_CATEGORIES)),
                }
            ),
        }
    )
    return datasets.DatasetInfo(
        description=_DESCRIPTION,
        features=features,
        homepage=_HOMEPAGE,
        license=_LICENSE,
    )

def _split_generators(self, dl_manager):
    """This method is tasked with downloading/extracting the data and defining the splits depending on the configuration"""

    downloaded_files = dl_manager.download_and_extract(_URLS)

    return [
        datasets.SplitGenerator(
            name=datasets.Split.TRAIN,
            # These kwargs will be passed to _generate_examples
            gen_kwargs={
                "filepath": os.path.join(downloaded_files["train"], "train.jsonl"),
                "split": "train"
            }
        ),
        datasets.SplitGenerator(
            name=datasets.Split.TEST,
            # These kwargs will be passed to _generate_examples
            gen_kwargs={
                "filepath": os.path.join(downloaded_files["test"], "test.jsonl"),
                "split": "test"
            }
        ),
    ]

# method parameters are unpacked from `gen_kwargs` as given in `_split_generators`
def _generate_examples(self, filepath, split):
    """
    This method handles input defined in _split_generators to yield (key, example) tuples from the dataset.
    The `key` is for legacy reasons (tfds) and is not important in itself, but must be unique for each example.
    """

    logger.info("generating examples from = %s", filepath)

    with open(filepath, encoding="utf-8") as f:
        for key, row in enumerate(f):
            data = json.loads(row)
            yield key, {
                "image_id": data["image_id"],
                # "image": data["image"],
                "width": data["width"],
                "height": data["height"],
                "file_name": data["file_name"],
                "url": data["url"],
                "objects": {
                    "id": data["objects"]["id"],
                    "area": data["objects"]["area"],
                    "bbox": data["objects"]["bbox"],
                    "category": [c-1 for c in data["objects"]["category"]]
                },
            }

In [48]:
_info(None)

DatasetInfo(description='This dataset contains 1028 images each 640x380 pixels.\nThe dataset is split into 249 test and 779 training examples.\nEvery image comes with MS COCO format annotations.\nThe dataset was collected in Carla Simulator, driving around in autopilot mode in various environments\n(Town01, Town02, Town03, Town04, Town05) and saving every i-th frame.\nThe labels where then automatically generated using the semantic segmentation information.\n', citation='', homepage='https://github.com/yunusskeete/Carla-COCO-Object-Detection-Dataset', license='MIT', features={'image_id': Value(dtype='int64', id=None), 'width': Value(dtype='int32', id=None), 'height': Value(dtype='int32', id=None), 'file_name': Value(dtype='string', id=None), 'url': Value(dtype='string', id=None), 'objects': Sequence(feature={'id': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'area': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'bbox': Sequence(feature=Val

In [49]:
from datasets import DownloadManager

dl_manager = DownloadManager()

train_generator, test_generator = _split_generators(None, dl_manager)
train_generator

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

SplitGenerator(name='train', gen_kwargs={'filepath': 'C:\\Users\\yunus\\.cache\\huggingface\\datasets\\downloads\\extracted\\1ea6e0a232639bfd8409d9e5ab3873942c85fd7c4696b5699e98577c07fd49cb\\train.jsonl', 'split': 'train'}, split_info=SplitInfo(name='train', num_bytes=0, num_examples=0, shard_lengths=None, dataset_name=None))

In [50]:
tuple(train_generator.gen_kwargs.values())

('C:\\Users\\yunus\\.cache\\huggingface\\datasets\\downloads\\extracted\\1ea6e0a232639bfd8409d9e5ab3873942c85fd7c4696b5699e98577c07fd49cb\\train.jsonl',
 'train')

In [51]:
train_egs = _generate_examples(None, train_generator.gen_kwargs["filepath"], train_generator.gen_kwargs["split"])
train_egs

<generator object _generate_examples at 0x000001D26C028F90>

In [52]:

next(train_egs)

(0,
 {'image_id': 0,
  'width': 640,
  'height': 380,
  'file_name': 'Town01_001020.png',
  'url': 'https://github.com/yunusskeete/Carla-COCO-Object-Detection-Dataset/raw/master/images/train/Town01_001020.png',
  'objects': {'id': [], 'area': [], 'bbox': [], 'category': []}})

In [53]:
datasets.load_dataset("C:/Users/yunus/Documents/Code/GitHub/avs/Carla-COCO-Object-Detection-Dataset/Carla-COCO-Object-Detection-Dataset-No-Images")

AttributeError: 'str' object has no attribute 'items'